<a href="https://colab.research.google.com/github/Voyageran/StartNN/blob/main/PaddingStride.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
from google.colab import drive
drive.mount('/content/gdrive')
sys.path.insert(0,"/content/content/notebooks/colabInstallPackage")

!cp -av '/content/gdrive/MyDrive/Colab Notebooks/d2l' '/content/'

Mounted at /content/gdrive
'/content/gdrive/MyDrive/Colab Notebooks/d2l' -> '/content/d2l'
'/content/gdrive/MyDrive/Colab Notebooks/d2l/paddle.py' -> '/content/d2l/paddle.py'
'/content/gdrive/MyDrive/Colab Notebooks/d2l/mxnet.py' -> '/content/d2l/mxnet.py'
'/content/gdrive/MyDrive/Colab Notebooks/d2l/tensorflow.py' -> '/content/d2l/tensorflow.py'
'/content/gdrive/MyDrive/Colab Notebooks/d2l/torch.py' -> '/content/d2l/torch.py'
'/content/gdrive/MyDrive/Colab Notebooks/d2l/__init__.py' -> '/content/d2l/__init__.py'
'/content/gdrive/MyDrive/Colab Notebooks/d2l/__pycache__' -> '/content/d2l/__pycache__'
'/content/gdrive/MyDrive/Colab Notebooks/d2l/__pycache__/mxnet.cpython-310.pyc' -> '/content/d2l/__pycache__/mxnet.cpython-310.pyc'
'/content/gdrive/MyDrive/Colab Notebooks/d2l/__pycache__/__init__.cpython-310.pyc' -> '/content/d2l/__pycache__/__init__.cpython-310.pyc'
'/content/gdrive/MyDrive/Colab Notebooks/d2l/__pycache__/torch.cpython-310.pyc' -> '/content/d2l/__pycache__/torch.cpython-

# **Padding and Strid**

Padding and Strid are both hyperparameters

## **Padding**

Convolution会让input进来的东西变小。更大的kernel可以更快地减小output size, i.e., $n_{h} \times n_{w}$ reduce to $(n_{h}-k_{h}+1) \times (n_{w}-k_{w}+1) $.如果不想让output很小

额外填充rows and columns such that input become larger.

E.g.,
**Original input**:
$
\begin{bmatrix}
0 & 1 & 2 \\
3 & 4 & 5 \\
6 & 7 & 8
\end{bmatrix}
$

Add extra pixels of filler around the boundary:
**Input**:
$
\begin{bmatrix}
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 2 & 0 \\
0 & 3 & 4 & 5 & 0 \\
0 & 6 & 7 & 8 & 0 \\
0 & 0 & 0 & 0 & 0
\end{bmatrix}
$


Add a total $p_{h}$ rows of padding and a total of $p_{w}$ columns of padding.

**Output size:**
$$
(n_{h}-k_{h}+p_{h}+1) \times (n_{w}-k_{w}+p_{w}+1)
$$

Often, we choose $p_{h} = k_{h}-1$, $p_{w} = k_{w}-1$, 这样刚好抵消掉大小。
- $k_{h}$ is an odd numebr: 上下两侧填充$p_{h}/2$
- $k_{h}$ is an even numebr: 在上侧填充$\ulcorner p_{h} /2 \urcorner$，在下侧填充$\llcorner p_{h} /2 \lrcorner$

In [11]:
import torch
from torch import nn

In [14]:
def comp_conv2d(conv2d, X):
  X = X.reshape((1,1) + X.shape)
  Y = conv2d(X)
  return Y.reshape(Y.shape[2:])

In [15]:
# E.g.,1
# 8-3+1=6 6+2=8(上下都有行)
conv2d = nn.Conv2d(1,1, kernel_size=3, padding = 1)
X = torch.rand(size = (8,8))
comp_conv2d(conv2d, X).shape
# X.reshape((1,1)+X.shape) #dim前面加入channel数, -> [[],X]

torch.Size([8, 8])

In [17]:
# E.g.2,
conv2d = nn.Conv2d(1,1,kernel_size=(5,3),padding = (2,1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

## **Stride**

给定kernel大小（如果kernel size过大，计算量会很大），用少量的层数去快速地reduce Input size。

Given height $s_{h}$ and width $s_{w}$, the output shape is
$$
\llcorner (n_{h}-k_{h}+p_{h}+s_{h}) / s_{h} \lrcorner \times \llcorner(n_{w}-k_{w}+p_{w}+s_{w}) / s_{w} \lrcorner
$$

Let $p_{h} = k_{h}-1$, $p_{w} = k_{w}-1$, then
$$
\llcorner (n_{h}+s_{h}-1) / s_{h} \lrcorner \times \llcorner(n_{w}+s_{w}-1) / s_{w} \lrcorner
$$

如果height和width可以被stride整除，则 $(n_{h}/s_{h}) \times (n_{w}/s_{w})$

In [20]:
import torch
from torch import nn

In [21]:
conv2d = nn.LazyConv2d(1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

In [22]:
conv2d = nn.LazyConv2d(1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])